In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [3]:
from my_utils.toy_data import invert_seq
train = invert_seq(5000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [4]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=64, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)
print(train_loader)

DataLoader(
	datasize: 5000
	batchsize: 64
	n_batches: 79
	trans_func: seq2seq
	shuffle: True
	device: cpu
)


In [25]:
from torch_models import AttnSeq2Seq, Seq2Seq

embed_size=64
dropout=0.3
model = Seq2Seq(embed_size=embed_size, hidden_size=embed_size, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    num_layers=1, bidirectional=True, dropout=dropout, rnn='lstm')
print(model)

Seq2Seq(
  (encoder): RNNEncoder(
    (embedding): Embedding(12, 64, padding_idx=11)
    (rnn): LSTM(64, 64, dropout=0.3, bidirectional=True)
  )
  (decoder): RNNEncoder(
    (embedding): Embedding(13, 64, padding_idx=12)
    (rnn): LSTM(64, 64, dropout=0.3)
  )
  (generator): MLP(
    (fc_out): Linear(in_features=64, out_features=12, bias=True)
    (dropout): Dropout(p=0.3)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
)


/Users/linghan/anaconda/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [26]:
%%time
from my_utils import Trainer, EvaluatorSeq, EvaluatorLoss
from my_utils.misc.logging import init_logger
from torch.optim import Adam, SGD

init_logger()

optimizer = Adam(model.parameters(), lr=0.001)
# optimizer = SGD(model.parameters(), lr=0.01)
evaluator = EvaluatorSeq(model, test_loader, measure='accuracy')
# evaluator = EvaluatorLoss(model, test_loader)

trainer = Trainer(model, train_loader)
trainer.train_epoch(optimizer, max_epoch=200,
              evaluator=evaluator, score_monitor=None)

[2018-10-19 00:59:40,046 INFO] epoch [1/200]	loss: 2.1912563133843337	
[2018-10-19 00:59:40,112 INFO] Evaluator accuracy: 0.1725	
[2018-10-19 00:59:42,177 INFO] epoch [2/200]	loss: 1.5989643093905872	
[2018-10-19 00:59:42,229 INFO] Evaluator accuracy: 0.3491271820448878	
[2018-10-19 00:59:44,290 INFO] epoch [3/200]	loss: 1.13802276759208	
[2018-10-19 00:59:44,360 INFO] Evaluator accuracy: 0.485	
[2018-10-19 00:59:46,738 INFO] epoch [4/200]	loss: 0.6854260314868975	
[2018-10-19 00:59:46,791 INFO] Evaluator accuracy: 0.7057356608478803	
[2018-10-19 00:59:49,185 INFO] epoch [5/200]	loss: 0.390911210941363	
[2018-10-19 00:59:49,242 INFO] Evaluator accuracy: 0.8575	


KeyboardInterrupt: 

In [9]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['1', '2', '7', '7']
['9', '3', '8', '4', '2']
['9', '7', '9']
['1', '8', '0', '2', '1']
['4', '6', '3']
['7', '5', '4', '4', '8']
['3', '9', '9', '3']
['8', '3', '7']
['9', '7', '2', '6']
['9', '3', '3', '8']
======= output ======
['7', '7', '2', '1']
['2', '4', '8', '3', '9']
['9', '7', '9']
['1', '2', '0', '8', '1']
['3', '6', '4']
['8', '4', '4', '5', '7']
['3', '9', '9', '3']
['7', '3', '8']
['6', '2', '7', '9']
['8', '3', '3', '9']
